#Practical example in Finance
Use Default load dataset, apply logistic regression, Naive Bayes classifier, Decision tree classifier and Support vector classifier to predict the varible **defaulted**

For each base model, create a bagging classifier and evalute the classifier in terms of an appropriate metric of performance.

deploy the best model using a microframework.

In [ ]:
from pandas import read_csv
data=read_csv('/content/Default_Loan (1).csv')
data=data.drop('Index', axis=1)
data.head()

y=data['Defaulted?']
X=data.drop('Defaulted?',axis=1)
data.head()

,Employed,Bank Balance,Annual Salary,Defaulted?
0,1,8754.36,532339.56,0
1,0,9806.16,145273.56,0
2,1,12882.60,381205.68,0
3,1,6351.00,428453.88,0
4,1,9427.92,461562.00,0


In [ ]:
from collections import Counter
Counter(y)

Counter({0: 9667, 1: 333})

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score

from sklearn.tree import DecisionTreeClassifier # decision tee algorithm for classification
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier 

In [ ]:
# get a list of models to evaluate
def get_models():
  models = dict() 
  models['bayes'] = GaussianNB()
  models['logistic']=LogisticRegression()
  models['dt_ent']=DecisionTreeClassifier(criterion='entropy')
  models['dt_gini']=DecisionTreeClassifier() # gini
  models['svc_lin'] = SVC(kernel='linear')
  models['svc_sig'] = SVC(kernel='sigmoid')
  models['svc_poly'] = SVC(kernel='poly')
  models['svc_rbf'] = SVC()

  return models

In [ ]:
models=get_models()
models.items()

dict_items([('bayes', GaussianNB()), ('logistic', LogisticRegression()), ('dt_ent', DecisionTreeClassifier(criterion='entropy')), ('dt_gini', DecisionTreeClassifier()), ('svc_lin', SVC(kernel='linear')), ('svc_sig', SVC(kernel='sigmoid')), ('svc_poly', SVC(kernel='poly')), ('svc_rbf', SVC())])

In [ ]:
# create the function to evaluate the models in terms of accuracy 
import numpy as np

def evaluate_model(model, X, y):
  acc = []
  for j in range(100):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state = j)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    acc.append(accuracy)
    scores=np.mean(acc) 
    return scores

In [ ]:
# create the function to evaluate the models in terms of accuracy 
import numpy as np

def evaluate_modelr(model, X, y):
  re = []
  for j in range(100):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state = j)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    recal = recall_score(y_test,y_pred)
    re.append(recal)
    Re=np.mean(re) 
    return Re

In [ ]:
from numpy import mean, std
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>bayes 0.969 (0.000)
>logistic 0.962 (0.000)
>dt_ent 0.954 (0.000)
>dt_gini 0.956 (0.000)
>svc_lin 0.966 (0.000)
>svc_sig 0.944 (0.000)
>svc_poly 0.962 (0.000)
>svc_rbf 0.962 (0.000)


In [ ]:
from numpy import mean, std
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_modelr(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>bayes 0.239 (0.000)
>logistic 0.000 (0.000)
>dt_ent 0.372 (0.000)
>dt_gini 0.345 (0.000)
>svc_lin 0.177 (0.000)
>svc_sig 0.027 (0.000)
>svc_poly 0.000 (0.000)
>svc_rbf 0.000 (0.000)


In [ ]:
!pip install imbalanced-learn

In [ ]:
import imblearn

In [ ]:
# define oversampling strategy
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')

 
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)

print(Counter(y_over))

Counter({0: 9667, 1: 9667})


In [ ]:
from numpy import mean, std
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X_over, y_over)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>bayes 0.878 (0.000)
>logistic 0.762 (0.000)
>dt_ent 0.986 (0.000)
>dt_gini 0.984 (0.000)
>svc_lin 0.874 (0.000)
>svc_sig 0.520 (0.000)
>svc_poly 0.824 (0.000)
>svc_rbf 0.880 (0.000)


In [ ]:
from numpy import mean, std
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_modelr(model, X_over, y_over)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>bayes 0.904 (0.000)
>logistic 0.860 (0.000)
>dt_ent 1.000 (0.000)
>dt_gini 1.000 (0.000)
>svc_lin 0.911 (0.000)
>svc_sig 0.475 (0.000)
>svc_poly 0.797 (0.000)
>svc_rbf 0.905 (0.000)


In [ ]:
# get a list of models to evaluate
def get_models_ens():
  models = dict() 

  models['dt_ent']=DecisionTreeClassifier(criterion='entropy')
  models['dt_bagging']=BaggingClassifier(base_estimator=models['dt_ent'], n_estimators=10) # gini

  return models

In [ ]:
models=get_models_ens()
models.items()

dict_items([('dt_ent', DecisionTreeClassifier(criterion='entropy')), ('dt_bagging', BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy')))])

In [ ]:
from numpy import mean, std
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_modelr(model, X_over, y_over)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>dt_ent 1.000 (0.000)
>dt_bagging 1.000 (0.000)


In [ ]:
from numpy import mean, std
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X_over, y_over)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>dt_ent 0.984 (0.000)
>dt_bagging 0.987 (0.000)


In [ ]:
# deployment phase

best = BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy'), n_estimators=10) # 10 bags
dt= best.fit(X_over,y_over) # fit the model
dt.predict(X_over.head(1))
X_over.head(1)

,Employed,Bank Balance,Annual Salary
0,1,8754.36,532339.56


In [ ]:
dt.predict(X_over.head(1))

array([0])

In [ ]:
x1=input('Employed:')  #SepalLengthCm	SepalWidthCm	PetalLengthCm	PetalWidthCm
x2=input('Bank Balance:')
x3=input('Annual Salary:')
x=np.array([x1,x2,x3])
y=dt.predict(x.reshape(1,-1))
print('recommendation:',y)